In [3]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0f}'.format)

In [4]:
def mic_equity_geog(df_in, equity_group):

    df_inc = df.pivot_table(
        index='person_work_mic',
        columns=equity_group,
        values='psexpfac',
        aggfunc='sum'
    ).reset_index()

    df_inc = df_inc.rename_axis(None, axis=1)
    pd.set_option('display.float_format', '{:,.0f}'.format)
    df_inc['Percent in Equity Geog'] = df_inc[1] / (df_inc[0] + df_inc[1])
    df_inc['Percent in Equity Geog'] = df_inc['Percent in Equity Geog'].fillna(0)
    df_inc['Percent in Equity Geog'] = (df_inc['Percent in Equity Geog'] * 100).round(1).astype(str) + '%'
    df_inc.sort_values(by=0, ascending=False)

    df_inc.rename(columns={'person_work_mic': 'MIC Work Location',
                        0: 'Does NOT Live in Equity Geog',
                        1: 'Lives in Equity Geog'}, inplace=True)
    df_inc.index = df_inc['MIC Work Location']
    df_inc.drop('MIC Work Location', axis=1, inplace=True)
    # Percent of people of color
    
    return df_inc

# Workers in Manufacturing-Industrial Centers (MICs)

In [5]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')

In [6]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
# df
df = df[df['pwtyp'].isin(['Paid Full-Time Worker', 'Paid Part-Time Worker'])]
df.loc[df['person_work_mic'].isnull(), 'person_work_mic'] = 'Outside MIC'
_df = df[['psexpfac','person_work_mic']].groupby('person_work_mic').sum()[['psexpfac']]
_df.index
_df.rename(columns={'psexpfac': 'Total Workers'}, inplace=True)
_df


,Total Workers
person_work_mic,
Ballard-Interbay,"15,816"
Cascade,"9,065"
Duwamish,"57,601"
Frederickson,"4,141"
Kent MIC,"42,531"
North Tukwila,"6,247"
Outside MIC,"2,083,779"
Paine Field / Boeing Everett,"35,117"
Port of Tacoma,"9,559"


In [9]:
mic_equity_geog(df, "hh_poverty_reg").loc[_df.index]

,Does NOT Live in Equity Geog,Lives in Equity Geog,Percent in Equity Geog
person_work_mic,,,
Ballard-Interbay,"11,407","4,409",27.9%
Cascade,"5,234","3,831",42.3%
Duwamish,"34,422","23,179",40.2%
Frederickson,"2,167","1,974",47.7%
Kent MIC,"21,174","21,357",50.2%
North Tukwila,"3,182","3,065",49.1%
Outside MIC,"1,299,210","784,569",37.7%
Paine Field / Boeing Everett,"20,204","14,913",42.5%
Port of Tacoma,"4,300","5,259",55.0%


In [10]:
mic_equity_geog(df, "hh_racial_reg").loc[_df.index]

,Does NOT Live in Equity Geog,Lives in Equity Geog,Percent in Equity Geog
person_work_mic,,,
Ballard-Interbay,"9,886","5,930",37.5%
Cascade,"7,970","1,095",12.1%
Duwamish,"26,057","31,544",54.8%
Frederickson,"2,253","1,888",45.6%
Kent MIC,"14,896","27,635",65.0%
North Tukwila,"2,272","3,975",63.6%
Outside MIC,"1,124,315","959,464",46.0%
Paine Field / Boeing Everett,"19,869","15,248",43.4%
Port of Tacoma,"4,555","5,004",52.3%


### English Proficiency

In [ ]:
mic_equity_geog(df, "hh_english_reg").loc[_df.index]

,Does NOT Live in Equity Geog,Lives in Equity Geog,Percent in Equity Geog
person_work_mic,,,
Ballard-Interbay,"10,641","4,722",30.7%
Cascade,"6,570","1,319",16.7%
Duwamish,"30,199","26,886",47.1%
Frederickson,"2,942",956,24.5%
Kent MIC,"16,973","25,029",59.6%
North Tukwila,"2,624","3,421",56.6%
Outside MIC,"1,303,020","785,747",37.6%
Paine Field / Boeing Everett,"17,545","16,213",48.0%
Port of Tacoma,"5,963","3,240",35.2%


# Commute Characteristics

In [ ]:
df = pd.read_csv(r'../../../../outputs/agg/dash/tour_mic_dest.csv')
# Commute tours to MICs
df = df[df['pdpurp']=="Work"]
df.loc[df['tour_d_mic'].isnull(), 'tour_d_mic'] = 'Outside MIC'

# tmodetp by tour_d_mic
df_mode = pd.pivot_table(
    df,
    index='tour_d_mic',
    columns='tmodetp',
    values='toexpfac',
    aggfunc='sum'
)

# mode share by tour_d_mic
df_mode = df_mode.div(df_mode.sum(axis=1), axis=0)
df_mode = df_mode.fillna(0)
df_mode = df_mode.reset_index()
df_mode.rename(columns={'tour_d_mic': 'MIC Work Location'}, inplace=True)
df_mode.index = df_mode['MIC Work Location']
df_mode.drop('MIC Work Location', axis=1, inplace=True)

df_mode['Transit'] = df_mode['Transit']+df_mode['Park']
df_mode['HOV'] = df_mode['HOV2'] + df_mode['HOV3+']
df_mode['Walk/Bike/Other'] =  df_mode['Walk']+df_mode['Bike']+df_mode['TNC']
# df_mode = df_mode.sort_values(by='Drive Alone', ascending=False)
# Show results as percentages
df_mode = df_mode.map(lambda x: f"{x:.1%}")
# df_mode.sort_values(by='Walk', ascending=False, inplace=True)


df_mode[['SOV', 'HOV', 'Transit', 'Walk/Bike/Other']]

tmodetp,SOV,HOV,Transit,Walk/Bike/Other
MIC Work Location,,,,
Ballard-Interbay,65.1%,26.0%,1.5%,7.4%
Cascade,66.4%,31.2%,0.2%,2.2%
Duwamish,65.9%,28.5%,3.0%,2.6%
Frederickson,67.8%,30.4%,0.0%,1.8%
Kent MIC,66.1%,31.0%,1.3%,1.6%
North Tukwila,67.6%,28.5%,1.5%,2.4%
Outside MIC,61.8%,27.7%,3.4%,7.2%
Paine Field / Boeing Everett,68.4%,29.4%,0.3%,2.0%
Port of Tacoma,66.1%,31.6%,0.7%,1.7%


In [ ]:
# Commute distance
pd.set_option('display.float_format', '{:,.1f}'.format)

df = pd.read_csv(r'../../../../outputs/agg/dash/tour_distance_mic.csv')
df = df[df['pdpurp'] == 'Work']
df['wt_dist'] = df['tautodist_bin'] * df['toexpfac']
df = df.groupby("person_work_mic").agg(
    {'wt_dist': 'sum', 'toexpfac': 'sum'}
)
df['average_distance'] = df['wt_dist']/df['toexpfac']
df[['average_distance']]

,average_distance
person_work_mic,
Ballard-Interbay,10.0
Cascade,11.6
Duwamish,12.6
Frederickson,9.0
Kent MIC,12.4
North Tukwila,12.6
Paine Field / Boeing Everett,10.9
Port of Tacoma,11.8
Puget Sound Industrial Center- Bremerton,12.4


# Demographics

In [ ]:
# Demographics 
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df['income_wt'] = df['hhincome_thousands'] * df['psexpfac']
df['income_wt'] = df['income_wt'].fillna(0)
df = df.groupby('person_work_mic').agg(
    {'psexpfac': 'sum', 'income_wt': 'sum'}
).reset_index()
df["avg_weighted_hh_income"] = df['income_wt']/df['psexpfac']
df['avg_weighted_hh_income'] = df['avg_weighted_hh_income'].apply(lambda x: f"${x:,.0f}")
df[['person_work_mic', 'avg_weighted_hh_income']]

,person_work_mic,avg_weighted_hh_income
0,Ballard-Interbay,"$189,771"
1,Cascade,"$152,312"
2,Duwamish,"$182,948"
3,Frederickson,"$139,220"
4,Kent MIC,"$164,763"
5,North Tukwila,"$167,601"
6,Paine Field / Boeing Everett,"$157,996"
7,Port of Tacoma,"$145,453"
8,Puget Sound Industrial Center- Bremerton,"$138,591"
9,Sumner Pacific,"$150,451"


In [ ]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df['hhsize_wt'] = df['hhsize'] * df['psexpfac']
df['hhsize_wt'] = df['hhsize_wt'].fillna(0)
df = df.groupby('person_work_mic').agg(
    {'psexpfac': 'sum', 'hhsize_wt': 'sum'}
)
df['avg_wt_hhsize'] = df['hhsize_wt']/df['psexpfac']
df = df.reset_index()
df[['person_work_mic', 'avg_wt_hhsize']]

,person_work_mic,avg_wt_hhsize
0,Ballard-Interbay,2.9
1,Cascade,3.5
2,Duwamish,3.1
3,Frederickson,3.6
4,Kent MIC,3.4
5,North Tukwila,3.3
6,Paine Field / Boeing Everett,3.2
7,Port of Tacoma,3.4
8,Puget Sound Industrial Center- Bremerton,3.1
9,Sumner Pacific,3.4


In [ ]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
# Pivot table of worker type by person_work_mic
df_worker_type = df.pivot_table(
    index='person_work_mic',
    columns='pwtyp',
    values='psexpfac',
    aggfunc='sum'
)

# Share by worker type

df_worker_type = df_worker_type.div(df_worker_type.sum(axis=1), axis=0)
df_worker_type.rename(columns={'Paid Full-Time Worker': 'Full Time', 'Paid Part-Time Worker': 'Part Time'})
df_worker_type.map(lambda x: f"{x:.1%}")

pwtyp,Paid Full-Time Worker,Paid Part-Time Worker
person_work_mic,,
Ballard-Interbay,82.5%,17.5%
Cascade,71.1%,28.9%
Duwamish,86.6%,13.4%
Frederickson,68.4%,31.6%
Kent MIC,84.9%,15.1%
North Tukwila,79.7%,20.3%
Paine Field / Boeing Everett,82.5%,17.5%
Port of Tacoma,83.3%,16.7%
Puget Sound Industrial Center- Bremerton,65.9%,34.1%
